### Fine-Tuning Nedir?

**Fine-tuning**, önceden eğitilmiş bir modeli (örneğin BERT), belirli bir görev (örneğin “ürün incelemesi sınıflandırma”) için küçük bir veri setiyle yeniden eğitmektir.

- **Neden gerekli?** Genel BERT modeli tüm dile dair bilgiyi içerir ama “telefon yorumları” gibi özel bir alanda yeterli doğruluk vermeyebilir.
- **Avantaj**: Sadece birkaç bin örnek ile yüksek performans elde edilebilir. Tam sıfırdan eğitmek yerine, mevcut bilgi üzerine inşa edilir.

---

### Fine-Tuning Süreci

1. **Veri Hazırlığı**: Etiketli veri (metin + sınıf) hazırlanır.
2. **Model Yükleme**: Önceden eğitilmiş BERT yüklenir.
3. **Sınıflandırıcı Katmanı Eklenir**: `[CLS]` çıktısına bağlı yeni bir katman.
4. **Eğitim**: Yeni katman ve BERT’in bazı katmanları küçük öğrenme oranı ile güncellenir.
5. **Değerlendirme**: Test verisi üzerinde doğruluk ölçülür.

Adım 1: Gerekli Kütüphaneleri Yükle

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
import torch

Adım 2: Örnek Veri Seti Oluştur

In [4]:
data = {
    "text": ["Harika bir ürün",
             "Kargo çok geç geldi",
             "İyi bir fiyat performans ürünü",
             "Kalitesiz malzeme kullanılmış",
             "Hızlı Teslimat,memnun kaldım",
             "Beklediğim gibi değil,çok pahalı"],
    "label": [1,0,1,0,1,0]
    
}
dataset = Dataset.from_dict(data)

Adım 3: Tokenizer ve Modeli Yükle

In [5]:
model = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model,num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Adım 4: Metinleri Tokenize Et

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,return_tensors=None)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Adım 5: Eğitim Ayarları ve Trainer

In [8]:
tranging_args = TrainingArguments(
    output_dir="./bertfinetuningornek",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=False,
)

trainer = Trainer(
    model=model,
    args=tranging_args,
    train_dataset=tokenized_datasets
)

Adım 6: Fine-Tuning’i Başlat

In [9]:
trainer.train()

/Users/ibrahimediz/anaconda3/envs/nlp2/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ibrahimediz/anaconda3/envs/nlp2/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss


/Users/ibrahimediz/anaconda3/envs/nlp2/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ibrahimediz/anaconda3/envs/nlp2/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Users/ibrahimediz/anaconda3/envs/nlp2/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ibrahimediz/anaconda3/envs/nlp2/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.

TrainOutput(global_step=3, training_loss=0.706830104192098, metrics={'train_runtime': 57.0869, 'train_samples_per_second': 0.315, 'train_steps_per_second': 0.053, 'total_flos': 4735998996480.0, 'train_loss': 0.706830104192098, 'epoch': 3.0})

Fine-Tuning Sonrası Kullanım

In [10]:
model.save_pretrained("./finetuned_bertfinetuningornek")
tokenizer.save_pretrained("./finetuned_bertfinetuningornek")


('./finetuned_bertfinetuningornek/tokenizer_config.json',
 './finetuned_bertfinetuningornek/special_tokens_map.json',
 './finetuned_bertfinetuningornek/vocab.txt',
 './finetuned_bertfinetuningornek/added_tokens.json',
 './finetuned_bertfinetuningornek/tokenizer.json')

In [ ]:
fine_tune_model = AutoModelForSequenceClassification.from_pretrained("./finetuned_bertfinetuningornek")
fine_tune_tokenizer = AutoTokenizer.from_pretrained("./finetuned_bertfinetuningornek")

yeni_yorum = "İyiymiş ürün"

inputs = tokenizer(yeni_yorum, return_tensors="pt",padding=True)

with torch.no_grad():
    output = fine_tune_model(**input)
    prediction = torch.softmax(output.logits,dim=-1)
    label = "Olumlu" if prediction[0][1] > 0.5 else "Olumsuz"

https://huggingface.co/docs/transformers/main_classes/trainer